# Lab - Parallel Computing Part 1 - GPU Kernel Implementation
## E6692 Spring 2022


In this lab we implement the layers of a Deep Learning model using CUDA. We use PyCUDA to configure the GPU context, define CUDA kernel functions, allocate memory, and transfer data between the CPU and the GPU. For more information on the GPU context, visit this [CUDA context documentation page](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#context).



In [ ]:
# import modules
import torch 
import torch.nn.functional as F
import numpy as np
import pycuda.gpuarray as gpuarray
import time

from utils.context import Context, GPUKernels

# define GPU
device = torch.device('cuda')

# define block size
BLOCK_SIZE = 32

# define kernel path
kernel_path = './kernels.cu'

# autoreload for utils files
%load_ext autoreload
%autoreload 2

%matplotlib inline

In Part 1 we write the kernel functions needed to implement a CUDA version of a CNN classifier. The CUDA language is an extension of C/C++ that allows us to program the GPU. See the [CUDA C++ Programming Guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#axzz4atgDRVPb) for detailed information regarding CUDA programming. We define the kernel functions in the file `kernels.cu`. There you will see the following functions:

* transpose() - example kernel function
* add() - **TODO**
* relu() - **TODO**
* conv2d() - **TODO**
* MaxPool2d() - **TODO**
* dot() - **TODO**

The CUDA kernel functions are initialized when an instance of the **Context()** class is created. You are encouraged to look through the code in **utils.context.py** to get a sense of how this process works, and to look at the block and grid dimension functions which are defined differently for 1 and 2 dimensions. After initialization, CUDA kernel functions can be used as PyCUDA functions. Below is a full example with `transpose()`.

In [ ]:
# initialize the GPU context and get source module
context = Context(BLOCK_SIZE)
source_module = context.getSourceModule(kernel_path)

In [ ]:
# get CUDA transpose function
transpose_cuda = source_module.get_function('transpose')

# initialize input and output
a = np.random.rand(50, 30).astype(np.float32)
b = np.zeros((a.shape[1], a.shape[0])).astype(np.float32)

# transfer data from CPU to GPU
a_d = gpuarray.to_gpu(a)
b_d = gpuarray.to_gpu(b)

# calculate block and grid dimensions
block_dims = context.block_dims
max_length = max(a.shape)
grid_dims = context.grid_dims((max_length, max_length))

# call CUDA function
transpose_cuda(a_d, b_d, np.int32(b.shape[0]), np.int32(b.shape[1]),
               block=block_dims, grid=grid_dims)

# transfer result from GPU --> CPU
b = b_d.get()

print("CUDA transpose matches PyTorch transpose: ", np.allclose(torch.t(torch.from_numpy(a)), b))

For more information check out [the PyCUDA documentation website](https://documen.tician.de/pycuda/). As is done above, we will compare our CUDA outputs to the outputs of corresponding PyTorch functions and layers.

We make a number of assumptions to simplify the kernel function implementations. For example, we restrict the mask count of convolutional layers to 1, assume that input shapes to each layer are such that padding is not required, and disregard the bias terms in convolutional layers. These assumptions are outlined in detail in the doc strings of the utils file functions. Pay careful attention to data types (**use FP32 precision**), 1D vs 2D grid and block dimensions, and tensor/array shapes between layers of the deep learning network. Also note that your CUDA function implementations will not be assessed for their time efficiency. The execution speed of a GPU kernel can vary dramatically depending on whether or not the programmer uses its memory heirarchy optimally. See this [NVIDIA blog](https://developer.nvidia.com/blog/using-shared-memory-cuda-cc/) about using shared memory for a more detailed description. You are encouraged, but not required, to use shared and constant memory where it makes sense. For example, the **transpose()** kernel loads matrix elements into the shared memory "block" using transposed indices before each thread writes an element into the transposed output matrix. This saves a significant amount of time as [accessing shared memory is 100 times slower than accessing global memory](https://stackoverflow.com/questions/14093692/whats-the-difference-between-cuda-shared-and-global-memory#:~:text=have%20in%20memory.-,Which%20is%20faster%20to%20access%3F,-In%20terms%20of).

## Part 1: Define GPU Kernel Functions for a Deep Learning Model


### add()

We will start with the simplest kernel function to implement: **add()**. The add function takes two arrays of the same shape and performs elementwise addition. Note that CUDA functions do not "return" values, they write the output result to a data structure pointed to by a function argument. In **add()** the result of the addition `A + B` should be written to the input array `A`. More explicitly, `A = A + B`.

To use the kernel function, you will also need to write PyCUDA code to allocate memory buffers and transfer data to and from the GPU. With the PyCUDA function implementation you are aiming to reproduce the functionality of [**torch.add()**](https://pytorch.org/docs/stable/generated/torch.add.html). Use the transpose function above as an example.

TODO: Complete the CUDA function **add()** in **kernels.cu**. Then complete **GPUKernels.add()** in **utils/context.py** and run the following cell to verify your implementation.

In [ ]:
cuda_functions = GPUKernels(context, source_module)

In [ ]:
################################################################
############### GPUKernels.add() TEST CASES ####################
################################################################

input_array_a_1 = np.random.randint(-10, high=10, size=32)
input_array_b_1 = np.random.randint(-10, high=10, size=32)

torch_input_a_1 = torch.from_numpy(input_array_a_1).unsqueeze(dim=0).unsqueeze(dim=0)
torch_input_b_1 = torch.from_numpy(input_array_b_1).unsqueeze(dim=0).unsqueeze(dim=0)

torch_output1 = torch.add(torch_input_a_1, torch_input_b_1)
cuda_output1 = cuda_functions.add(input_array_a_1, input_array_b_1)

print("GPUKernels.add() Test 1 passed: ", np.allclose(torch_output1, cuda_output1))

input_array_a_2 = np.random.randint(-1000, high=-40, size=(435, 34))
input_array_b_2 = np.random.randint(-10, high=10, size=(435, 34))

torch_input_a_2 = torch.from_numpy(input_array_a_2).unsqueeze(dim=0).unsqueeze(dim=0)
torch_input_b_2 = torch.from_numpy(input_array_b_2).unsqueeze(dim=0).unsqueeze(dim=0)

torch_output2 = torch.add(torch_input_a_2, torch_input_b_2)
cuda_output2 = cuda_functions.add(input_array_a_2, input_array_b_2)

print("GPUKernels.add() Test 2 passed: ", np.allclose(torch_output2, cuda_output2))

input_array_a_3 = np.random.randint(1000, high=10000, size=1000)
input_array_b_3 = np.random.randint(0, high=10, size=1000)

torch_input_a_3 = torch.from_numpy(input_array_a_3).unsqueeze(dim=0).unsqueeze(dim=0)
torch_input_b_3 = torch.from_numpy(input_array_b_3).unsqueeze(dim=0).unsqueeze(dim=0)

torch_output3 = torch.add(torch_input_a_3, torch_input_b_3)
cuda_output3 = cuda_functions.add(input_array_a_3, input_array_b_3)

print("GPUKernels.add() Test 2 passed: ", np.allclose(torch_output3, cuda_output3))

################################################################
################ DO NOT MODIFY THE CODE ABOVE ##################
################################################################

### relu()

Next we implement the [ReLU activation](https://www.kaggle.com/dansbecker/rectified-linear-units-relu-in-deep-learning) function **relu()**. This is a popular nonlinear activation function used in deep learning networks. See [torch.nn.ReLU()](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html#torch.nn.ReLU) for more details.

Just like with **add()**, you will also need to write PyCUDA code to allocate memory buffers and transfer data to and from the GPU. 

TODO: Complete the CUDA function **relu()** in **kernels.cu**. Then complete **GPUKernels.relu()** in **utils/context.py** and run the following cell to verify your implementation.

In [ ]:
################################################################
############### GPUKernels.relu() TEST CASES ###################
################################################################

input_array1 = np.random.randint(-3, high=3, size=(32, 32)).astype(np.float32)
torch_input1 = torch.from_numpy(input_array1).unsqueeze(dim=0)

torch_output1 = F.relu(torch_input1)
cuda_output1 = cuda_functions.relu(input_array1)

print("GPUKernels.relu() Test 1 passed: ", np.allclose(torch_output1, cuda_output1))

input_array2 = np.random.randint(0, high=1, size=(400, 400)).astype(np.float32)
torch_input2 = torch.from_numpy(input_array2).unsqueeze(dim=0)

torch_output2 = F.relu(torch_input2)
cuda_output2 = cuda_functions.relu(input_array2)

print("GPUKernels.relu() Test 2 passed: ", np.allclose(torch_output2, cuda_output2))

input_array3 = np.random.randint(-100, high=90, size=1234).astype(np.float32)
torch_input3 = torch.from_numpy(input_array3).unsqueeze(dim=0)

torch_output3 = F.relu(torch_input3)
cuda_output3 = cuda_functions.relu(input_array3)

print("GPUKernels.relu() Test 3 passed: ", np.allclose(torch_output3, cuda_output3))

################################################################
################ DO NOT MODIFY THE CODE ABOVE ##################
################################################################

### conv2d()

Next we implement the 2D convolution function **conv2d()**. There are multiple ways to write the 2D convolution CUDA kernel using shared, global, and constant memory. For this assignment, any of these implementations will do. We make several assumptions to simplify the implementation in CUDA. In reference to the PyTorch versions of this function, [**torch.nn.functional.conv2d()**](https://pytorch.org/docs/stable/generated/torch.nn.functional.conv2d.html#torch.nn.functional.conv2d) and [**torch.nn.Conv2d()**](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html), we assume a square 2D input (H, W) where $H == W$ rather than a 4D input (N, C, H, W), `padding='same'`, `bias=False`, `stride=1`, and a square and odd kernel/mask shape.

In the PyCUDA code you will need to allocate memory for the input array, the weight array (kernel/mask), and the output array.

TODO: Complete the CUDA function **conv2d()** in **kernels.cu**. Then complete **GPUKernels.conv2d()** in **utils/context.py** and run the following cell to verify your implementation.

In [ ]:
################################################################
############### GPUKernels.conv2d() TEST CASES #################
################################################################

input_array1 = np.random.randint(-10, high=10, size=(32, 32))
mask1 = np.random.randint(-10, high=10, size=(3, 3))

torch_input1 = torch.from_numpy(input_array1).unsqueeze(dim=0).unsqueeze(dim=0)
torch_mask1 = torch.from_numpy(mask1).unsqueeze(dim=0).unsqueeze(dim=0)

torch_output1 = F.conv2d(torch_input1, torch_mask1, padding='same')
cuda_output1 = cuda_functions.conv2d(input_array1, mask1)

print("GPUKernels.conv2d() Test 1 passed: ", np.allclose(torch_output1, cuda_output1))

input_array2 = np.random.randint(-10, high=10, size=(33, 33))
mask2 = np.random.randint(-10, high=10, size=(11, 11))

torch_input2 = torch.from_numpy(input_array2).unsqueeze(dim=0).unsqueeze(dim=0)
torch_mask2 = torch.from_numpy(mask2).unsqueeze(dim=0).unsqueeze(dim=0)

torch_output2 = F.conv2d(torch_input2, torch_mask2, padding='same')
cuda_output2 = cuda_functions.conv2d(input_array2, mask2)

print("GPUKernels.conv2d() Test 2 passed: ", np.allclose(torch_output2, cuda_output2))

input_array3 = np.random.randint(-10, high=10, size=(1000, 1000))
mask3 = np.random.randint(-10, high=10, size=(3, 3))

torch_input3 = torch.from_numpy(input_array3).unsqueeze(dim=0).unsqueeze(dim=0)
torch_mask3 = torch.from_numpy(mask3).unsqueeze(dim=0).unsqueeze(dim=0)

torch_output3 = F.conv2d(torch_input3, torch_mask3, padding='same')
cuda_output3 = cuda_functions.conv2d(input_array3, mask3)

print("GPUKernels.conv2d() Test 3 passed: ", np.allclose(torch_output3, cuda_output3))

################################################################
################ DO NOT MODIFY THE CODE ABOVE ##################
################################################################

### MaxPool2d()

Next we implement the 2D Max Pool function **MaxPool2d()**. See [**torch.nn.MaxPool2d**](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html) for details. Once again, we make simplifying assumptions. We require a square pooling kernel, kernel_size to be equal to stride, and a square input array. 

TODO: Complete the CUDA function **MaxPool2d()** in **kernels.cu**. Then complete **GPUKernels.MaxPool2d()** in **utils/context.py** and run the following cell to verify your implementation.

In [ ]:
################################################################
############# GPUKernels.MaxPool2d() TEST CASES ################
################################################################

input_array1 = np.random.randint(0, high=2, size=(32, 32)).astype(np.float32)
kernel_size1 = 3
torch_MaxPool2d = torch.nn.MaxPool2d(kernel_size1, stride=kernel_size1)
torch_input1 = torch.from_numpy(input_array1).unsqueeze(dim=0)

torch_output1 = torch_MaxPool2d(torch_input1)
cuda_output1 = cuda_functions.MaxPool2d(input_array1, kernel_size=kernel_size1)

print("GPUKernels.MaxPool2d() Test 1 passed: ", np.allclose(torch_output1, cuda_output1))

input_array2 = np.random.randint(-1000, high=1000, size=(1290, 1290)).astype(np.float32)
kernel_size2 = 9
torch_MaxPool2d = torch.nn.MaxPool2d(kernel_size2, stride=kernel_size2)
torch_input2 = torch.from_numpy(input_array2).unsqueeze(dim=0)

torch_output2 = torch_MaxPool2d(torch_input2)
cuda_output2 = cuda_functions.MaxPool2d(input_array2, kernel_size=kernel_size2)

print("GPUKernels.MaxPool2d() Test 2 passed: ", np.allclose(torch_output2, cuda_output2))

input_array3 = np.random.randint(-1, high=0, size=(4, 4)).astype(np.float32)
kernel_size3 = 4
torch_MaxPool2d = torch.nn.MaxPool2d(kernel_size3, stride=kernel_size3)
torch_input3 = torch.from_numpy(input_array3).unsqueeze(dim=0)

torch_output3 = torch_MaxPool2d(torch_input3)
cuda_output3 = cuda_functions.MaxPool2d(input_array3, kernel_size=kernel_size3)

print("GPUKernels.MaxPool2d() Test 3 passed: ", np.allclose(torch_output3, cuda_output3))

################################################################
################ DO NOT MODIFY THE CODE ABOVE ##################
################################################################

### linear()

TODO: Complete the CUDA function **dot()** in **kernels.cu**. You may find it useful (or necessary) to create test cases for your **dot()** implementation. You are not required to generate test cases, but if you choose to, compare your implementation to [**torch.mm()**](https://pytorch.org/docs/stable/generated/torch.mm.html) in the cell below. 

In [ ]:
# Cell for dot() vs torch.mm() test cases, if needed



To create a dense, fully connected neural network layer, we combine several operations to apply a linear transformation:

$$y = x \cdot w^T + b$$

where $x$ is the input matrix, $w$ is the weight matrix, $b$ is the bias matrix, and $y$ is the output of the transformation.

TODO: After implementing **dot()** in **kernels.cu**, complete **GPUKernels.linear()** in **utils/context.py**. The **linear()** function combines the **transpose()**, **dot()**, and **add()** CUDA kernels to create a linear, fully connected layer of a neural network. We assume the input matrix $x$ to be one dimensional (1, N), the weight matrix of shape (M, N), and the bias matrix of shape (1, M). The Run the following cell to verify your linear implementation.

In [ ]:
# TODO: think about the bias shape

################################################################
############## GPUKernels.linear() TEST CASES ##################
################################################################

input_array1 = np.random.randint(-10, high=255, size=(409,)).astype(np.float32)
weights1 = np.random.randint(0, high=12, size=(190, 409)).astype(np.float32)
bias1 = np.random.randint(-10, high=10, size=(190,)).astype(np.float32)

torch_input1 = torch.from_numpy(input_array1).unsqueeze(dim=0).unsqueeze(dim=0)
torch_weights1 = torch.from_numpy(weights1)
torch_bias1 = torch.from_numpy(bias1)

torch_output1 = F.linear(torch_input1, torch_weights1, bias=torch_bias1)
cuda_output1 = cuda_functions.linear(input_array1, weights1, bias1)

print("GPUKernels.linear() Test 1 passed: ", np.allclose(torch_output1, cuda_output1))

input_array2 = np.random.randint(0, high=255, size=(49,))
weights2 = np.random.randint(0, high=10, size=(120, 49))
bias2 = np.random.randint(-10, high=10, size=(120,))

torch_input2 = torch.from_numpy(input_array2).unsqueeze(dim=0).unsqueeze(dim=0)
torch_weights2 = torch.from_numpy(weights2)
torch_bias2 = torch.from_numpy(bias2)

torch_output2 = F.linear(torch_input2, torch_weights2, bias=torch_bias2)
cuda_output2 = cuda_functions.linear(input_array2, weights2, bias2)

print("GPUKernels.linear() Test 2 passed: ", np.allclose(torch_output2, cuda_output2))

input_array3 = np.random.randint(0, high=100, size=(128,))
weights3 = np.random.randint(-10, high=10, size=(256, 128))
bias3 = np.random.randint(-10, high=100, size=(256,))

torch_input3 = torch.from_numpy(input_array3).unsqueeze(dim=0).unsqueeze(dim=0)
torch_weights3 = torch.from_numpy(weights3)
torch_bias3 = torch.from_numpy(bias3)

torch_output3 = F.linear(torch_input3, torch_weights3, bias=torch_bias3)
cuda_output3 = cuda_functions.linear(input_array3, weights3, bias3)

print("GPUKernels.linear() Test 3 passed: ", np.allclose(torch_output3, cuda_output3))

################################################################
################ DO NOT MODIFY THE CODE ABOVE ##################
################################################################